This notebook processes the output data from Assignment3_Q2.doe

In [1]:
#Import libraries

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from scipy import stats
import pandas as pd
from scipy.stats import lognorm
from scipy.stats import norm

----------------------

In [2]:
#Import data

dataAll=pd.read_csv('Q2/CompletionTimes.txt', sep=' ', header=None)
dataAll=dataAll.iloc[:, 0:5]
dataAll.columns=['A', 'C', 'G', 'J', 'CompletionTimes']
display(dataAll.shape)
display(dataAll.head(5))

(200, 5)

,A,C,G,J,CompletionTimes
0,1.764977,4.390893,4.376375,1.341751,11.873996
1,1.451379,6.223080,6.021862,1.924595,15.620915
2,1.506632,1.361793,4.905874,2.009787,9.784085
3,1.883346,8.586010,6.616497,2.440421,19.526273
4,2.414775,7.485873,5.427642,2.239814,17.568104


In [3]:
#Select first 50 replications for computing optimal lambda
data=dataAll.iloc[0:50]

In [18]:
#Calculate correlations between processes on critical path and completion times

display('Correlation coefficients')
display(np.corrcoef(data.A, data.CompletionTimes))
display(np.corrcoef(data.C, data.CompletionTimes))
display(np.corrcoef(data.G, data.CompletionTimes))
display(np.corrcoef(data.J, data.CompletionTimes))

'Correlation coefficients'

array([[1.        , 0.19186006],
       [0.19186006, 1.        ]])

array([[1.       , 0.9140907],
       [0.9140907, 1.       ]])

array([[1.        , 0.46571112],
       [0.46571112, 1.        ]])

array([[1.        , 0.38788862],
       [0.38788862, 1.        ]])

#### Process C shows strongest correlation with completion time so choose this as covariate

In [17]:
display('Covariance matrix')
display(np.round(np.cov(data.C, data.CompletionTimes), 3))

'Covariance matrix'

array([[3.345, 3.72 ],
       [3.72 , 4.951]])

In [15]:
#Compute lambda_star
lambda_star=np.cov(data.C, data.CompletionTimes)[0, 1]/np.cov(data.C, data.CompletionTimes)[0, 0]
display('lambda_star='+str(np.round(lambda_star, 3)))

'lambda_star=1.112'

#### Compute W for remaining  150 replications 

In [9]:
dataCalc=dataAll.iloc[50:, :]
mean_C=np.mean(dataCalc["C"])
dataCalc["W"]=dataCalc.loc[:,'CompletionTimes']-lambda_star*(dataCalc.loc[:,'C']-mean_C)
display(dataCalc.describe())


/Applications/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,A,C,G,J,CompletionTimes,W
count,150.000000,150.000000,150.000000,150.000000,150.000000,150.000000
mean,1.969486,5.071796,5.160275,2.105702,14.314505,14.314505
std,0.422610,1.577553,0.847928,0.384975,1.892026,1.030178
min,1.045500,1.704568,3.244754,1.042270,10.613402,12.233658
25%,1.628566,3.965660,4.662687,1.840351,12.883201,13.609580
50%,1.967996,4.989166,5.148251,2.094254,14.370261,14.328213
75%,2.237165,6.352303,5.783974,2.358937,15.742872,15.029403
max,2.933144,8.474129,6.880413,2.926643,18.752012,16.871010


In [13]:
#Compute CI
n=len(dataCalc)
sd_W=np.std(dataCalc["W"], ddof=1)
sd_C=np.std(dataCalc["C"], ddof=1)
CI_hw=stats.t.ppf(1-0.025, n-1)*sd_W/np.sqrt(n)

CI_L=np.mean(dataCalc["CompletionTimes"])-CI_hw
CI_U=np.mean(dataCalc["CompletionTimes"])+CI_hw
CI=(CI_L, CI_U)
display('CI:'+str(np.round(CI, 3)))
display('sd C:'+str(np.round(sd_C, 3))+'. sd W: '+str(np.round(sd_W, 3)))

'CI:[14.148 14.481]'

'sd C:1.578. sd W: 1.03'